# Game prototype

In [ ]:
from collections.abc import Callable
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import List, Union, TypeVar, Tuple, Set, Type

from collections import Counter
from IPython.display import Image, clear_output

import logging
import regex

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    filename="app.log",
    filemode="a",
)

CanvasType = Union[Image, str]
QueryType = Union[
    CanvasType, List[CanvasType]
]  # change query to canvas in interactions
AlphabetType = List[QueryType]  # TODO: ask Niklas???

In [ ]:
# ───────────────────────────────────
# EXAMPLE & TASK
# ───────────────────────────────────


@dataclass
class Example:
    """
    One example puzzle: contains the message to encode,
    the allowed alphabets for encoding, and the alphabet for decoding.
    """

    def __init__(
        self,
        message: QueryType,
        encoder_alphabet: QueryType,
        decoder_alphabet: QueryType,
    ):
        self.message = message
        self.encoder_alphabet = encoder_alphabet  # TODO ask niklas, if alphabet_name is at task do examples need alphabets
        self.decoder_alphabet = decoder_alphabet


@dataclass()
class Task:
    """A set of examples forming a complete task."""

    def __init__(
        self,
        examples: List[Example],
        separator: chr,
        description_format: Tuple[str, str, str],
    ):
        self.examples = examples
        self.separator = separator
        (
            self.message_category,
            self.encoder_alphabet_name,
            self.decoder_alphabet_name,
        ) = description_format
        self.encode_description = "\nGiven a {message_category} as query, encode it with symbols out of {encoder_alphabet_name}(alphabet). Divide symbols by: '{separator}'".format(
            message_category=self.message_category,
            encoder_alphabet_name=self.encoder_alphabet_name,
            separator=self.separator,
        )
        self.decode_description = "\nGiven a cipher as query decode it back to a {message_category} with {decoder_alphabet_name}(alphabet). Divide symbols by: '{separator}".format(
            message_category=self.message_category,
            decoder_alphabet_name=self.decoder_alphabet_name,
            separator=self.separator,
        )
        self.validator_description = ""
        # TODO. ask niklas if sepreator is alwys betwenn chars cause if "" is sepratot emojis and chars are complicated    promlem(if no seprator but alphabeat contain heart and bb)


@dataclass()
class RoundState:
    """
    Stores the intermediate and final results of a single example:
    the cipher and the decoded message(message prime).
    Class used for logging
    """

    def __init__(
        self, example: Example
    ):  # TODO: use, should prevent  seperate alphabet description querey trnsport
        self.example = example
        self.cipher = None
        self.ciphers = []
        self.message_prime = None
        self.message_primes = []


# ───────────────────────────────────
# Signature
# ───────────────────────────────────


class Signature(ABC):
    """Abstract base for all signatures."""

    @abstractmethod
    def check(
        self,
        query: QueryType,
        alphabet: AlphabetType,
    ) -> bool:
        """Check whether the query is valid given the alphabet."""
        pass


class WordsSignature(Signature, ABC):
    """Signature allowing arbitrary combinations of symbols."""

    NAME = "Valid is any word over the alphabet"
    pass


class NWordsSignature(Signature, ABC):
    """Signature restricting words to maximum length n."""

    NAME = "Valid is any word over the alphabet of length n"
    pass


class SingleChoiceSignature(Signature, ABC):
    """Signature restricted to a single symbol."""

    NAME = "Valid is any single choice of the alphabet"
    pass


class MultipleChoiceSignature(Signature, ABC):
    """Signature restricted to a fixed number of symbols."""

    NAME = "Valid are choices of the alphabet"
    pass


class PermutationsSignature(Signature, ABC):
    """Signature restricted to permutation of the entire alphabet."""

    NAME = "Valid is a permutation of the alphabet"
    pass


class VariationSignature(Signature, ABC):
    """Signature allowing variation of the alphabet."""

    NAME = "Valid is a variation of the alphabet"
    pass


# ───────────────────────────────────
# Adapter
# ───────────────────────────────────


class Adapter(ABC):
    """Interface for human or AI adapter interacting with the game."""

    @abstractmethod
    def present_data(
        self,
        query: QueryType,
        alphabet: AlphabetType,
        name: str,
    ) -> None:
        """Display the query and alphabet to the adapter."""
        pass

    @abstractmethod
    def get_data(self) -> QueryType:
        """Return the adapter's response (cipher or decoded message)."""
        pass


class HumanAdapter(Adapter):
    """adapter interacting with a human user."""

    pass


class AIAdapter(Adapter):
    """adapter interacting with an AI model."""

    pass


# ───────────────────────────────────
# Agent
# ───────────────────────────────────


class Agent(ABC):
    """Handles encoding and communication with an adapter."""

    def __init__(self, signature_cls: Type[Signature], adapter_cls: Type[Adapter]):
        self.signature = signature_cls()
        self.adapter = adapter_cls()

    @abstractmethod
    def ask_adapter_for_query(
        self, query: QueryType, alphabet: AlphabetType
    ) -> QueryType:
        """Run the adapter to produce a query."""
        pass

    @abstractmethod
    def verify_query_with_signature(
        self, query: QueryType, alphabet: AlphabetType
    ) -> bool:
        """Check if the produced query is valid."""
        pass


# ───────────────────────────────────
# GAME
# ───────────────────────────────────


class Game(ABC):
    """Game logic handling transmission, validation, and communication."""

    def __init__(
        self,
        task: Task,
        signature_encoder_cls: Tuple[Type[Signature], Type[Adapter]],
        signature_decoder_cls: List[Tuple[Type[Signature], Type[Adapter]]],
        agent_cls: Type[Agent],
    ):
        self.task = task

        self.agent_sender = agent_cls(*signature_encoder_cls)

        self.agent_receivers = [agent_cls(*t) for t in signature_decoder_cls]

        self.points = {r: 0 for r in self.agent_receivers}

    @abstractmethod
    def call_agent(
        self, agent: Agent, query: QueryType, alphabet: AlphabetType
    ) -> Tuple[bool, QueryType]:
        """Calls agent, returns a  Tuple of is_valid and query"""
        pass

    @abstractmethod
    def play_example(self, example: Example):
        pass

    @abstractmethod
    def play_task(self):
        """Play all examples in the task."""
        pass

In [ ]:
# -----------------------------
# Signature
# -----------------------------


class MyWordsSignature(WordsSignature):
    def check(self, query, alphabet) -> bool:
        query = (
            query.strip().replace("\n", "").replace("\r", "").split("-")
        )  # TODO - seperator
        return all(c in alphabet for c in query)


# -----------------------------
# Adapter
# -----------------------------


class MyHumanAdapter(HumanAdapter):
    def present_data(self, query, alphabet, name, description):
        # clear_output()
        print(description)
        print(f"\n{name}\nQuery: {query}\nAlphabet: {alphabet}")

    def get_data(self):
        query = input("\nEnter: ")
        print(f"\nYou typed: {query}")
        # time.sleep(3)
        return query  # todo: und man muss das auch noch dem adapter sagen. auch erlauben, einen index anzugeben? -> alle 3 über die Task lösen. TODO: ask niklas


# -----------------------------
# Agent
# -----------------------------


class MyAgent(Agent):

    def __init__(self, signature_cls, adapter_cls):
        super().__init__(signature_cls, adapter_cls)

    def ask_adapter_for_query(self, query, alphabet, description):
        self.adapter.present_data(
            query, alphabet, type(self.signature).NAME, description
        )
        return self.adapter.get_data()

    def verify_query_with_signature(self, query, alphabet):
        return self.signature.check(query, alphabet)


# -----------------------------
# Game
# -----------------------------


class MyGame(Game):  # TODO: ask Niklas, logic thats not game dependent to abstract

    def __init__(self, task, signature_encoder_cls, signature_decoder_cls, agent_cls):
        super().__init__(task, signature_encoder_cls, signature_decoder_cls, agent_cls)

        # For simplicity, assume only one receiver # TODO: remove
        self.agent_receiver = self.agent_receivers[0]

    def call_agent(self, agent, query, alphabet, description):
        query = agent.ask_adapter_for_query(query, alphabet, description)
        success = agent.verify_query_with_signature(query, alphabet)

        return success, query

    def play_example(self, example):

        success, cipher = self.call_agent(
            agent=self.agent_sender,
            query=example.message,
            alphabet=example.encoder_alphabet,
            description=self.task.encode_description,
        )
        while not success:
            print("\nRetry")
            success, cipher = self.call_agent(
                self.agent_sender,
                example.message,
                example.encoder_alphabet,
                description=self.task.encode_description,
            )

        print("\nNext Phase")

        success, message_prime = self.call_agent(
            self.agent_receiver,
            cipher,
            example.decoder_alphabet,
            description=self.task.decode_description,
        )
        while not success:
            print("\nRetry")
            success, message_prime = self.call_agent(
                self.agent_receiver,
                cipher,
                example.decoder_alphabet,
                description=self.task.decode_description,
            )

        print("\nNext Round")

        points = int(example.message.lower() == message_prime.lower())
        self.points[self.agent_receiver] += points

        print(f"\nreceived points: {points}")

    def play_task(self):
        for example in self.task.examples:
            self.play_example(example)

        print(f"\ntotal points: {self.points[self.agent_receiver]}")

In [ ]:
if __name__ == "__main__":
    emoji_alphabet = [
        "❄️",
        "🛳️",
        "❤️",
        "💥",
        "🌊",
        "🧊",
        "💀",
        "🦈",
        "🚢",
        "🌹",
        "a",
    ]  # a debugging symbol
    letter_alphabet = list("abcdefghijklmnopqrstuvwxyz")
    separator = "-"
    description_format = ("movie title", "emoji list", "lower case letters")

    examples = [
        Example("Titanic", emoji_alphabet, letter_alphabet),
        # Example("Inception", emoji_alphabet, letter_alphabet),
        # Example("Avatar", emoji_alphabet, letter_alphabet),
    ]
    task = Task(examples, separator, description_format)

    game = MyGame(
        task=task,
        signature_encoder_cls=(MyWordsSignature, MyHumanAdapter),
        signature_decoder_cls=[(MyWordsSignature, MyHumanAdapter)],
        agent_cls=MyAgent,
    )

    game.play_task()


Given a movie title as query, encode it with symbols out of emoji list(alphabet). Divide symbols by: '-'

Valid is any word over the alphabet
Query: Titanic
Alphabet: ['❄️', '🛳️', '❤️', '💥', '🌊', '🧊', '💀', '🦈', '🚢', '🌹', 'a']

You typed: a

Next Phase

Given a cipher as query decode it back to a movie title with lower case letters(alphabet). Divide symbols by: '-

Valid is any word over the alphabet
Query: a
Alphabet: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

You typed: a

Next Round

received points: 0

total points: 0


In [ ]:
# TODO:
# second game (4 Bilder, 1 Wort)
# human verifier, ask niklas
# code to .py, except main
# fix different signatures

In [ ]:
# ──────────────────────────────
# N Words


class MyWordsEncoder(WordsEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols)


# ──────────────────────────────
# N Words
# ──────────────────────────────


class MyNWordsEncoder(NWordsEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols) and len(symbols) <= n


# -----------------------------
# SingleChoice
# -----------------------------
class MySingleChoiceEncoder(SingleChoiceEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols) and len(symbols) == 1


# -----------------------------
# MultipleChoice
# -----------------------------
class MyMultipleChoiceEncoder(MultipleChoiceEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols)


# -----------------------------
# Permutation
# -----------------------------
class MyPermutationsAllEncoder(PermutationsAllEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return sorted(symbols) == sorted(encoder_alphabet)


# -----------------------------
# Variation
# -----------------------------
class MyPermutationsSubEncoder(PermutationsSubEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        c1 = Counter(cipher)
        c2 = Counter(encoder_alphabet)
        return all(c1[x] <= c2[x] for x in c1) and len(c1) == len(encoder_alphabet)